# Projet Classifier automatiquement des biens de consommation : Notebook des requêtes API

In [1]:
!pip install requests
import requests
import pandas as pd


## Testons la collecte des produits à base de champagne
Résultat : 4 produits trouvés

In [2]:
# Les identifiants API
app_id = '4ec4b99f'
app_key = 'bc5a7f695c217b8c544058287fecd9c0'

# URL de l'API Edamam pour les aliments
url = 'https://api.edamam.com/api/food-database/v2/parser'

# Paramètres de la requête pour extraire les produits à base de champagne
params = {
    'ingr': 'champagne',
    'app_id': app_id,
    'app_key': app_key,
    'category': 'generic-foods'
}

all_products = []

# Faire la requête initiale
response = requests.get(url, params=params)

# Vérifier que la requête s'est bien passée
if response.status_code == 200:
    data = response.json()
    all_products.extend(data.get('hints', []))  # Ajouter les produits à la liste

    # Pagination : continuer à récupérer les résultats suivants
    while 'more' in data and data['more']:
        params['page'] = params.get('page', 1) + 1  # Passer à la page suivante
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            all_products.extend(data.get('hints', []))
        else:
            print(f"Erreur lors de la requête à la page {params['page']} : {response.status_code}")
            break
else:
    print(f"Erreur lors de la requête initiale : {response.status_code}")

# Afficher le nombre total de produits trouvés
print(f"Nombre total de produits à base de champagne trouvés : {len(all_products)}")

# Afficher les informations de chaque produit
for product in all_products:
    food_info = product['food']
    food_id = food_info.get('foodId', '')
    label = food_info.get('label', '')
    category = food_info.get('category', '')
    food_contents_label = food_info.get('foodContentsLabel', '')
    image = food_info.get('image', '')

    print(f"ID: {food_id}, Label: {label}, Category: {category}, Contents: {food_contents_label}, Image: {image}")

Nombre total de produits à base de champagne trouvés : 4
ID: food_a656mk2a5dmqb2adiamu6beihduu, Label: Champagne, Category: Generic foods, Contents: , Image: https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg
ID: food_bncple4a2uagu1b4hov92budz2vs, Label: Champagne Grape, Category: Generic foods, Contents: , Image: https://www.edamam.com/food-img/ca5/ca55ac74deb991d159942c65777115df.jpg
ID: food_bxdqsxkax2vgmpbv8e4ygb6zfnkn, Label: Champagne Vinegar, Category: Generic foods, Contents: , Image: https://www.edamam.com/food-img/5f6/5f69b84c399d778c4728e9ab4f8065a2.jpg
ID: food_brtd7beak7p11iastcbh1a6wyz3q, Label: Sherry or Champagne Vinegar, Category: Generic foods, Contents: , Image: https://www.edamam.com/food-img/5f6/5f69b84c399d778c4728e9ab4f8065a2.jpg


## Extraction des 10 premiers produits contenant les données suivantes :foodId, label, category, foodContentsLabel, image.

In [3]:
# Paramètres de la requête pour extraire les produits
params = {
    'ingr': 'food',  # Requête plus générique pour obtenir des produits variés
    'app_id': app_id,
    'app_key': app_key,
    'category': 'generic-foods'
}

# Faire la requête
response = requests.get(url, params=params)

# Vérifier que la requête s'est bien passée
if response.status_code == 200:
    data = response.json()
    foods = data['hints'][:10]  # Extraire les 10 premiers produits

    if not foods:
        print("Aucun produit trouvé.")
    else:
        # Créer une liste pour les lignes du CSV
        csv_data = []

        # Extraire les informations nécessaires
        for food in foods:
            food_info = food['food']
            food_id = food_info.get('foodId', '')
            label = food_info.get('label', '')
            category = food_info.get('category', '')
            food_contents_label = food_info.get('foodContentsLabel', '')
            image = food_info.get('image', '')

            # Ajouter les informations à la liste
            csv_data.append([food_id, label, category, food_contents_label, image])

        # Créer un DataFrame avec les données
        df = pd.DataFrame(csv_data, columns=['foodId', 'label', 'category', 'foodContentsLabel', 'image'])
        
        # Afficher le DataFrame
        display(df)

        # Écrire les données dans un fichier CSV
        csv_filename = 'first_10_food_products.csv'
        df.to_csv(csv_filename, index=False)

        print(f"Les données des 10 premiers produits ont été enregistrées dans {csv_filename}.")
else:
    print(f"Erreur lors de la requête : {response.status_code}")

,foodId,label,category,foodContentsLabel,image
0,food_aai0cf1b1twew9bybx0kdam9pd8c,Food Coloring,Generic foods,,https://www.edamam.com/food-img/5dd/5dd9d13618...
1,food_a76bh12br5qrptbnnne27af21k11,Food Thickener,Generic foods,,https://www.edamam.com/food-img/714/714be7b219...
2,food_akjf74ibll2id8aqb8xbha4t4axl,Oats (Includes Foods for USDA's Food Distribut...,Generic foods,,
3,food_bchehm9bnczd4pbm2xybdasc7d21,"Fast Foods, Cheeseburger",Generic foods,,https://www.edamam.com/food-img/8cc/8cc981eb92...
4,food_adfqt94am41aytbiwrlrzav5vltg,"Fast Foods, Coleslaw",Generic foods,,https://www.edamam.com/food-img/4f5/4f5797bc4a...
5,food_bfor9p4amjokjla6t0z42agkrbb5,"Fast Food, Biscuit",Generic foods,,https://www.edamam.com/food-img/3dd/3ddf57cf3d...
6,food_a1bmw3xb7iaforay591owa8g177u,Fast Food Brownie,Generic foods,,
7,food_ayssvlkbrtwk3za1p4macbp73itx,"Fast Foods, Hamburger",Generic foods,,https://www.edamam.com/food-img/319/319ee2fa02...
8,food_a2b3q33bnje4iual2pba2at89tzi,"Cheese, Cheddar (Includes Foods for USDA's Foo...",Generic foods,,
9,food_be6ctzna8a8u4wa9hlv3na5qcvpk,Angel Food Cake,Generic foods,,https://www.edamam.com/food-img/e57/e57060f752...


Les données des 10 premiers produits ont été enregistrées dans first_10_food_products.csv.
